In [1]:
from ultralytics import YOLO
model = YOLO("modelFolder/model_v2.pt")
results = model(source="validimage/1_Color.png",device="cpu", project = "results",name="test6d", conf=0.8 )
for result in results:
    image_points = result.masks.xyn
    print(len(image_points))
    


image 1/1 c:\Team_Project-1\validimage\1_Color.png: 384x640 1 box, 418.6ms
Speed: 15.6ms preprocess, 418.6ms inference, 15.6ms postprocess per image at shape (1, 3, 384, 640)


1


In [2]:
from crop_ply import *
xyz_output= cropPly(image_points,"validimage/1.ply")
print(xyz_output)

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.
[PointCloud with 1514 points.]


In [6]:
import open3d as o3d
import numpy as np

# 포인트 클라우드를 시각화
for cloud in xyz_output:
    # 포인트 클라우드의 경계 상자 계산
    aabb = cloud.get_axis_aligned_bounding_box()
    aabb.color = (1, 0, 0)

    obb = cloud.get_oriented_bounding_box()
    obb.color = (0, 1, 0)

    # 클라우드 포인트의 중심 위치 계산
    center = np.mean(np.asarray(cloud.points), axis=0)
    print(center)

    # 중앙 위치에 좌표 프레임 메시 생성
    mesh = o3d.geometry.TriangleMesh.create_coordinate_frame(size=0.07)  # size를 조절하여 좌표 프레임 크기 설정 가능

    # 중앙 위치에 좌표 프레임 메시를 이동
    mesh.translate(center)

    # 시각화
    o3d.visualization.draw_geometries([cloud, aabb, obb, mesh])

    # PCA를 수행하여 주요 주성분 벡터를 찾습니다.
    points = np.asarray(cloud.points)
    covariance_matrix = np.cov(points, rowvar=False)
    eigenvalues, eigenvectors = np.linalg.eigh(covariance_matrix)

    # 주요 주성분 벡터와 각각의 축과의 각도를 계산합니다.
    x_axis = np.array([1, 0, 0])  # x 축
    y_axis = np.array([0, 1, 0])  # y 축
    z_axis = np.array([0, 0, 1])  # z 축

    angle_x_rad = np.arccos(np.dot(eigenvectors[:, -1], x_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(x_axis)))
    angle_x_deg = np.degrees(angle_x_rad)

    angle_y_rad = np.arccos(np.dot(eigenvectors[:, -1], y_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(y_axis)))
    angle_y_deg = np.degrees(angle_y_rad)

    angle_z_rad = np.arccos(np.dot(eigenvectors[:, -1], z_axis) / (np.linalg.norm(eigenvectors[:, -1]) * np.linalg.norm(z_axis)))
    angle_z_deg = np.degrees(angle_z_rad)

    print(f"요피치 롤(x 축 기준): {angle_x_deg}도")
    print(f"요피치 롤(y 축 기준): {angle_y_deg}도")
    print(f"요피치 롤(z 축 기준): {angle_z_deg}도")
    
    o3d.io.write_point_cloud("robot/test.ply", cloud)


[    0.47369     0.44579    -0.64438]
요피치 롤(x 축 기준): 93.35221844791774도
요피치 롤(y 축 기준): 176.24944284696656도
요피치 롤(z 축 기준): 88.31986441282403도


In [7]:
import open3d as o3d
import numpy as np

# 표준화된 데이터를 가지고 있는 PCD 파일을 불러옵니다.
loaded_cloud = o3d.io.read_point_cloud("robot/test.ply")

# 포인트 클라우드 좌표를 가져옵니다.
points = np.asarray(loaded_cloud.points)

# x 및 y 좌표를 원래 데이터 범위로 역변환합니다.
x_min = np.min(points[:, 0])
y_min = np.min(points[:, 1])
z_min = np.min(points[:, 2])

x_range = np.max(points[:, 0]) - x_min
y_range = np.max(points[:, 1]) - y_min
z_range = np.max(points[:, 2]) - z_min

original_x = x_min + points[:, 0] * x_range
original_y = y_min + points[:, 1] * y_range
original_z = z_min + points[:, 2] * z_range  # z 좌표의 역변환

# 역변환된 좌표를 포인트 클라우드에 적용합니다.
points[:, 0] = original_x
points[:, 1] = original_y
points[:, 2] = original_z

# 포인트 클라우드를 업데이트합니다.
loaded_cloud.points = o3d.utility.Vector3dVector(points)

# 업데이트된 포인트 클라우드를 시각화합니다.
o3d.visualization.draw_geometries([loaded_cloud],width=640, height=360)
o3d.io.write_point_cloud("robot/bigtest.ply", loaded_cloud)


True